<a href="https://colab.research.google.com/github/chayaphon/Data_Wrangling/blob/main/Wrangling_Code/data_cleansing_Pastic_cards_usage.ipynb#scrollTo=IzFf1CXgipHb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

# Define sheet name
FILE_NAME = '../Sources/Data.xlsx'

# Get all sheet name in Excel file
sheet_name_list = pd.ExcelFile(FILE_NAME).sheet_names

# Show all sheet name and index number
for index, sheet_name in enumerate(sheet_name_list):
    print(f"{index} : {sheet_name}")

# Read data base on selected SELECT_SHEET_NAME
SELECT_SHEET_NAME_1 = sheet_name_list[1]
SELECT_SHEET_NAME_2 = sheet_name_list[2]

# add dataframe for Volume and Value sheet
df_volume = pd.read_excel(FILE_NAME, sheet_name= SELECT_SHEET_NAME_1, skiprows = 5)
df_value = pd.read_excel(FILE_NAME, sheet_name= SELECT_SHEET_NAME_2, skiprows = 5)

0 : Index
1 : Vol. of Plastic card usage
2 : Value. of Plastic card usage
3 : Mobile,internet banking
4 : Promptpay


In [2]:
### Data Cleansing ###

# --- Volume ---#
# Change column name and drop un-related column
df_volume.rename(columns = {"Unnamed: 1" : "Attribute"}, inplace = True)
df_volume.dropna(subset = ["Attribute"], inplace = True)
df_volume.drop(columns = ["Unnamed: 0"], inplace = True)
df_volume.replace("n.a.", np.nan, inplace = True)

# Filter out last row of column 'Attribute'
df_volume = df_volume[df_volume['Attribute'] != "Total"]

# Cleansing Column name (trim and remove leeter after year)
col_list_clean_volume = []
col_list_trim_volume = [col_name.strip() for col_name in df_volume.columns]
for col_name in col_list_trim_volume:
    if len(col_name.split(" ")) <= 2:
        col_list_clean_volume.append(col_name)
    else:
        cleaned_col_name =  " ".join(col_name.split(" ")[:-1])
        col_list_clean_volume.append(cleaned_col_name)
        
# Overwrite existing column with new columns clean column    
df_volume.columns = col_list_clean_volume


# --- Value ---#
# Change column name and drop un-related column
df_value.rename(columns = {"Unnamed: 1" : "Attribute"}, inplace = True)
df_value.dropna(subset = ["Attribute"], inplace = True)
df_value.drop(columns = ["Unnamed: 0"], inplace = True)
df_value.replace("n.a.", np.nan, inplace = True)

# Filter out last row of column 'Attribute'
df_value = df_value[df_value['Attribute'] != "Total"]

# Cleansing Column name (trim and remove leeter after year)
col_list_clean_volume = []
col_list_trim_volume = [col_name.strip() for col_name in df_volume.columns]
for col_name in col_list_trim_volume:
    if len(col_name.split(" ")) <= 2:
        col_list_clean_volume.append(col_name)
    else:
        cleaned_col_name =  " ".join(col_name.split(" ")[:-1])
        col_list_clean_volume.append(cleaned_col_name)
        
# Overwrite existing column with new columns clean column    
df_value.columns = col_list_clean_volume


In [3]:
### Data Cleansing 2 ###
# Remove /Number from 'Attribute' columns
text_clean_list = []
for text in list(df_volume['Attribute']):
    if text.split(" ")[-1].endswith("/"):
        text_clean_list.append(" ".join(text.split(" ")[:-1]))
    else:
        text_clean_list.append(text)

In [4]:
### Data Shaping1 ###
# Seperate each attribute as parent and sub-topic

result_text_list = []
Parent = []

for index in range(len(text_clean_list)):
    list_text = text_clean_list[index].split("   ")
    
    if len(list_text) == 1:
        Parent = [list_text[0].strip()]
        result_text_list.append(" ::: ".join(Parent))
    else:
        Parent = Parent[:len(list_text)-1]
        Parent.append(list_text[len(list_text)-1].strip())
        result_text_list.append(" ::: ".join(Parent))
        
# Assign new value to 'Attribute' column 
df_volume['Attribute'] = result_text_list
df_value['Attribute'] = result_text_list

# Expland Attribute into multiple columns depend on their hireachy
prefix_col = 'Attribute_'
df_volume = df_volume['Attribute'].str.split(" ::: ", expand = True).add_prefix(prefix_col).join(df_volume)
df_value = df_value['Attribute'].str.split(" ::: ", expand = True).add_prefix(prefix_col).join(df_value)

# drop Attribute column
df_volume.drop(columns = ["Attribute"], inplace = True)
df_value.drop(columns = ["Attribute"], inplace = True)


In [5]:
### Data Shaping 2 ###
# remove head(total) row

# get attribute list
selected_col = [col for col in df_volume if col.startswith('Attribute_')]

# identify head(total) row
Row_to_Remove = []
for i in range(len(selected_col)):
    Attribute = 'Attribute_' + str(i)
    for j in range(len(df_volume)):
        
        cur_val = df_volume[Attribute][j]
        if j < len(df_volume)-1:
            next_val = df_volume[Attribute][j+1]
        else:
            next_val = ""
        if j > 0:
            prev_val = df_volume[Attribute][j-1]
        else:
            prev_val = ""
    
        if cur_val != None and cur_val != prev_val and cur_val == next_val:
            Row_to_Remove.append(j)

# remove row head(total) row
df_volume.drop(labels=Row_to_Remove,axis=0,inplace=True)
df_value.drop(labels=Row_to_Remove,axis=0,inplace=True)

# reset row index
df_volume.reset_index(inplace=True,drop=True)
df_value.reset_index(inplace=True,drop=True)

In [6]:
### Data Shaping 3 ###

# --- Volumn --- #
# Unpivot MonthYear Column for Volumn 
level_col_list = [col_name for col_name in df_volume.columns if col_name.startswith(prefix_col)]
value_col_list = [col_name for col_name in df_volume.columns if col_name not in level_col_list]
df_volume = pd.melt(df_volume, id_vars = level_col_list, value_vars = value_col_list)

# add column Measre
df_volume['MeasureName'] = 'volume_k'


# --- Value --- #
# Unpivot MonthYear Column for Volumn 
level_col_list = [col_name for col_name in df_value.columns if col_name.startswith(prefix_col)]
value_col_list = [col_name for col_name in df_value.columns if col_name not in level_col_list]
df_value = pd.melt(df_value, id_vars = level_col_list, value_vars = value_col_list)

# add column Measre
df_value['MeasureName'] = 'value_b'


In [7]:
### Data Shaping 4 ###
# Union Volume(df_volume) and Value(df_value)
df_final = pd.concat([df_volume,df_value])

# rename column
df_final.rename(columns = {"Attribute_0" : "ChannelType" , "Attribute_1" : "UsageType"}, inplace = True)
df_final.rename(columns = {"variable" : "MonthYear" , "value" : "M_Value"}, inplace = True)

df_final['TransType'] = 'Plastic cards'

In [8]:
# save output to csv
df_final.to_csv(f'../Clean/Pastic_cards_usage_Clean.csv', index = False)
